# German Used Cars on eBay
INTRODUCTION

## Data
The dataset found on [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/downloads/used-cars-database.zip/3) contains over 370000 used cars scraped with Scrapy from Ebay-Kleinanzeigen. The content of the data is in german, so one has to translate it first if one can not speak german. Those fields are included: autos.csv:

- **dateCrawled** : when this ad was first crawled, all field-values are taken from this date
- **name** : "name" of the car
- **seller** : private or dealer
- **offerType**
- **price** : the price on the ad to sell the car
- **abtest**
- **vehicleType**
- **yearOfRegistration** : at which year the car was first registered
- **gearbox**
- **powerPS** : power of the car in PS
- **model**
- **kilometer** : how many kilometers the car has driven
- **monthOfRegistration** : at which month the car was first registered
- **fuelType**
- **brand**
- **notRepairedDamage** : if the car has a damage which is not repaired yet
- **dateCreated** : the date for which the ad at ebay was created
- **nrOfPictures** : number of pictures in the ad (unfortunately this field contains everywhere a 0 and is thus useless (bug in crawler!) )
- **postalCode**
- **lastSeenOnline** : when the crawler saw this ad last online

The fields lastSeen and dateCreated could be used to estimate how long a car will be at least online before it is sold.

In [1]:
import pandas as pd
import numpy as np

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [3]:
print(autos.info())
autos.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


The dataset contains 20 columns most of which are strings, and some of the columns contain null values, but none with more than about 20% null values so we should be able to work with most of the columns.

Let's convert columns named in CamelCase to Python's snake_case.

In [4]:
autos.rename(columns={'yearOfRegistration': 'registration_year',
                      'monthOfRegistration': 'registration_month',
                      'notRepairedDamage': 'unrepaired_damage',
                      'dateCreated': 'ad_created',
                      'dateCrawled': 'date_crawled',
                      'offerType': 'offer_type',
                      'vehicleType': 'vehicle_type',
                      'postalCode': 'postal_code',
                      'lastSeen': 'last_seen'},
                       inplace=True)
print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'powerPS', 'model',
       'kilometer', 'registration_month', 'fuelType', 'brand',
       'unrepaired_damage', 'ad_created', 'nrOfPictures', 'postal_code',
       'last_seen'],
      dtype='object')


Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for text columns where all or almost all values are the same, since these typically don't have useful information for analysis.

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,kilometer,registration_month,fuelType,brand,unrepaired_damage,ad_created,nrOfPictures,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


In [6]:
print(autos['seller'].value_counts())
print()
print(autos['offer_type'].value_counts())

privat        371525
gewerblich         3
Name: seller, dtype: int64

Angebot    371516
Gesuch         12
Name: offer_type, dtype: int64


It looks like nrOfPictures has a value of zero for all rows as noted in column descriptions at the top of the Data section, so it can be safely dropped as it has no information.

Additionally, the seller and offerType columns are nearly all one value so let's get rid of those as well.  

In [7]:
autos.drop(['nrOfPictures', 'seller', 'offer_type'], axis=1, inplace=True)

## Price
Now let's more in depth at the price column.

In [8]:
print(autos['price'].describe())
autos['price'].value_counts().head(20)

count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64


0       10778
500      5670
1500     5394
1000     4649
1200     4594
2500     4438
600      3819
3500     3792
800      3784
2000     3432
999      3364
750      3203
650      3150
4500     3053
850      2946
2200     2936
700      2936
1800     2886
900      2874
950      2793
Name: price, dtype: int64

We can see that there are a large number of cars being listed for $0, but since this is not realistic we should remove them.

In [9]:
autos['price'].value_counts().sort_index(ascending=True).head(10)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
Name: price, dtype: int64

In [10]:
autos['price'].value_counts().sort_index(ascending=False).head(50)

2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
2795000        1
1600000        2
1300000        1
1250000        2
1234566        1
1111111        2
1010010        1
1000000        5
999999        13
999990         1
911911         1
849000         1
820000         1
780000         1
745000         2
725000         1
700000         1
650000         1
619000         1
600000         2
599000         1
585000         1
579000         1
517895         1
500000         2
488997         1
487000         1
485000         1
466000         1
445000         1
440000         1
420000         1
399997         1
395000         1
390000         1
Name: price, dtype: int64

Given that eBay is an auction site, there could legitimately be items where the opening bid is 1 dollar so we will keep the 1 dollar items, but remove anything above \$390,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [11]:
autos = autos[autos['price'].between(1, 390000)]

## Dates
Right now the date_crawled, ad_created, and last_seen columns are stored in Pandas as strings. We'll change them to datetime objects removing the time of day to make them easier to work with.

In [12]:
date_cols = ['date_crawled', 'ad_created', 'last_seen']
autos[date_cols] = pd.to_datetime(autos[date_cols].stack()).dt.date.unstack()

Now let's look at what the most common dates for each of these three columns is.

In [13]:
autos['date_crawled'].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025546
2016-03-06    0.014483
2016-03-07    0.035656
2016-03-08    0.033468
2016-03-09    0.034114
2016-03-10    0.032648
2016-03-11    0.032772
2016-03-12    0.036241
2016-03-13    0.015783
2016-03-14    0.036330
2016-03-15    0.033424
2016-03-16    0.030205
2016-03-17    0.031649
2016-03-18    0.013118
2016-03-19    0.035271
2016-03-20    0.036399
2016-03-21    0.035681
2016-03-22    0.032492
2016-03-23    0.032002
2016-03-24    0.029914
2016-03-25    0.032800
2016-03-26    0.031974
2016-03-27    0.030227
2016-03-28    0.035063
2016-03-29    0.034126
2016-03-30    0.033535
2016-03-31    0.031874
2016-04-01    0.034145
2016-04-02    0.035096
2016-04-03    0.038812
2016-04-04    0.037628
2016-04-05    0.012780
2016-04-06    0.003128
2016-04-07    0.001617
Name: date_crawled, dtype: float64

In [14]:
autos['ad_created'].value_counts(normalize=True, dropna=False).sort_index()

2014-03-10    0.000003
2015-03-20    0.000003
2015-06-11    0.000003
2015-06-18    0.000003
2015-08-07    0.000003
2015-08-10    0.000003
2015-09-04    0.000006
2015-09-09    0.000003
2015-10-14    0.000003
2015-11-02    0.000003
2015-11-08    0.000003
2015-11-10    0.000003
2015-11-12    0.000003
2015-11-13    0.000003
2015-11-17    0.000003
2015-11-23    0.000006
2015-11-24    0.000006
2015-12-05    0.000008
2015-12-06    0.000003
2015-12-17    0.000003
2015-12-27    0.000003
2015-12-28    0.000003
2015-12-30    0.000008
2016-01-02    0.000011
2016-01-03    0.000003
2016-01-06    0.000003
2016-01-07    0.000008
2016-01-08    0.000003
2016-01-10    0.000011
2016-01-13    0.000008
                ...   
2016-03-09    0.034162
2016-03-10    0.032523
2016-03-11    0.032806
2016-03-12    0.036078
2016-03-13    0.017056
2016-03-14    0.035265
2016-03-15    0.033349
2016-03-16    0.030427
2016-03-17    0.031247
2016-03-18    0.014022
2016-03-19    0.034303
2016-03-20    0.036488
2016-03-21 

In [15]:
autos['last_seen'].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001264
2016-03-06    0.004098
2016-03-07    0.005202
2016-03-08    0.007939
2016-03-09    0.009824
2016-03-10    0.011460
2016-03-11    0.012955
2016-03-12    0.023239
2016-03-13    0.008410
2016-03-14    0.012176
2016-03-15    0.016324
2016-03-16    0.016418
2016-03-17    0.028702
2016-03-18    0.006888
2016-03-19    0.016329
2016-03-20    0.019884
2016-03-21    0.020026
2016-03-22    0.020508
2016-03-23    0.018015
2016-03-24    0.019163
2016-03-25    0.019000
2016-03-26    0.015958
2016-03-27    0.016720
2016-03-28    0.022188
2016-03-29    0.023284
2016-03-30    0.023725
2016-03-31    0.024243
2016-04-01    0.023896
2016-04-02    0.024967
2016-04-03    0.025311
2016-04-04    0.025535
2016-04-05    0.126964
2016-04-06    0.218950
2016-04-07    0.130435
Name: last_seen, dtype: float64

It looks like they skew heavily towards the later dates in the range (04/05 to 04/07), which makes sense for eBay where ads will be taken down as the cars are sold or the ad expires.

The year the car was made is an important metric so let's get a better understanding of the distribution for these values.

In [16]:
autos['registration_year'].describe()

count    360639.000000
mean       2004.433098
std          81.016536
min        1000.000000
25%        1999.000000
50%        2004.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

After using the describe method, we can see that the average year of registration is 2004, with most being between 1999 and 2008. Additionally, there are some dirty values around the min and max where users likely entered incorrect dates.

We should take a closer look at the listings that fall outside of the 1900 - 2020 interval and see if it's safe to remove these rows entirely.

In [17]:
before_1900_or_after_2020 = (autos['registration_year'] < 1900) | (autos['registration_year'] > 2020)
autos.loc[before_1900_or_after_2020, 'registration_year'].value_counts()

1000    32
9999    18
5000    17
3000     6
6000     6
1800     5
7000     4
9000     4
1234     4
1500     3
2500     3
4000     3
8000     2
4500     2
5555     2
5911     2
8888     2
1300     2
1600     2
1111     1
8500     1
2066     1
6200     1
5900     1
1602     1
8200     1
4100     1
7500     1
2900     1
2290     1
3700     1
7777     1
7100     1
9450     1
1001     1
1255     1
5600     1
3800     1
4800     1
5300     1
6500     1
2222     1
2200     1
7800     1
3200     1
1400     1
2800     1
1039     1
Name: registration_year, dtype: int64

Given that none of these dates make sense--cars were invented in the early 20th century and the other dates are in the future--and there are relatively few of them we can safely get rid of these rows.

In [18]:
autos.drop(autos.index[before_1900_or_after_2020], axis=0, inplace=True)
autos['registration_year'].value_counts(normalize=True)

2000    0.064140
1999    0.061114
2005    0.060265
2006    0.055494
2001    0.054773
2003    0.054387
2004    0.054018
2002    0.052207
2007    0.048562
1998    0.047785
2008    0.044481
2009    0.042991
1997    0.038916
2010    0.034034
2011    0.033277
1996    0.028877
2017    0.027729
2012    0.025962
2016    0.025565
1995    0.024963
2013    0.016980
2014    0.013229
1994    0.012888
2018    0.010580
1993    0.009149
2015    0.008097
1992    0.008033
1991    0.007371
1990    0.006943
1989    0.003492
          ...   
2019    0.000047
1953    0.000047
1954    0.000039
1937    0.000033
1952    0.000031
1938    0.000022
1929    0.000022
1936    0.000019
1947    0.000014
1933    0.000014
1934    0.000014
1935    0.000014
1943    0.000014
1945    0.000014
1931    0.000011
1941    0.000011
1932    0.000011
1939    0.000011
1930    0.000011
1949    0.000008
1948    0.000008
1923    0.000008
1928    0.000006
1927    0.000006
1940    0.000006
1944    0.000006
1946    0.000006
1942    0.0000

## Brand Prices
Let's get an idea of the average price for the most popular car brands on the German eBay.

In [21]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.213553
bmw               0.108582
opel              0.107387
mercedes_benz     0.095991
audi              0.088712
ford              0.068909
renault           0.048340
peugeot           0.029981
fiat              0.025962
seat              0.018949
skoda             0.015465
mazda             0.015368
smart             0.014353
citroen           0.014034
nissan            0.013601
toyota            0.012827
hyundai           0.009961
sonstige_autos    0.009437
mini              0.009312
volvo             0.008982
mitsubishi        0.008219
honda             0.007584
kia               0.006927
suzuki            0.006316
alfa_romeo        0.006275
porsche           0.006022
chevrolet         0.004971
chrysler          0.003828
dacia             0.002469
jeep              0.002180
daihatsu          0.002150
land_rover        0.002103
subaru            0.002080
jaguar            0.001687
daewoo            0.001479
saab              0.001440
trabant           0.001376
l

German brands (VW, BMW, Mercedes, etc) represent nearly half of all the listings, and Volkswagen alone has double the cars for sale as each of the next two brands.

Since the brands are so top heavy, we'll only look at the average price for the top 5.

In [31]:
top_5_brands = autos['brand'].value_counts().head(5).index
autos[autos['brand'].isin(top_5_brands)].groupby('brand')['price'].mean()

brand
audi             8964.038680
bmw              8370.696089
mercedes_benz    8473.402150
opel             2940.832998
volkswagen       5322.489959
Name: price, dtype: float64

There are distinct price groups among the top 5 brands:
- Audi, BMW, and Mercedes are the most expensive
- Opel is the least expensive
- Volkswagen is in the middle between the luxury and cheap brands